Get Src

In [30]:
import os
import numpy as np

df_captcha = "Src_CaptchaIMG"

captchas = [
    os.path.join(df_captcha, i) for i in os.listdir(df_captcha)
]

number_of_pic = len(captchas)

print(number_of_pic)

1070


preprocess_CAPTCHA

In [31]:
import cv2


def preprocess_CAPTCHA(img):
    """Takes a CAPTCHA image and thresholds it."""
    gray_Scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_with_border = cv2.copyMakeBorder(gray_Scale, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    
    preprocessed = cv2.threshold(
        gray_with_border, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )[1]
    
    return gray_with_border, preprocessed

get_CAPTCHA_label

In [32]:
def get_CAPTCHA_label(path_to_file):
    """Get the CAPTCHA text from the file name."""
    filename = os.path.basename(path_to_file)
    
    label = filename.split(".")[0]
    
    return label


find_bounding_rectangles_of_contours

In [33]:
def find_bounding_rectangles_of_contours(contours):
    """Determines the bounding rectangles of the contours of the cropped letters."""
    letter_bounding_rectangles = []
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_bounding_rectangles.append((x, y, half_width, h))
            letter_bounding_rectangles.append((x + half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append((x, y, w, h ))
            
    return letter_bounding_rectangles

CAPTCHA_to_gray_scale_and_bounding_rectangle

In [34]:
def CAPTCHA_to_gray_scale_and_bounding_rectangles(captcha_image_file):
    """Take a CAPTCHA and output a grayscale version as well as the bounding rectangles of its cropped letters."""
    image = cv2.imread(captcha_image_file)
    gray, preprocessed = preprocess_CAPTCHA(image)
    
    contours = cv2.findContours(
        preprocessed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    contours = contours[0]
    
    letter_bounding_rectangles = find_bounding_rectangles_of_contours(contours)
    
    letter_bounding_rectangles = sorted(letter_bounding_rectangles, key=lambda x: x[0])
    
    return gray, letter_bounding_rectangles

bounding_rectangle_to_letter_image

In [35]:
def bounding_rectangle_to_letter_image(letter_bounding_box, grayscaled):
    """Obtains the letter defined by a bounding box."""
    x, y, w, h  = letter_bounding_box
    
    letter_image = grayscaled[y - 2 : y + h + 2, x - 2 : x + w + 2]
    
    return letter_image

extracted

In [36]:
captcha_processing_output_folder = "extractedIMG"

character_counts = {}


def crop_bounding_rectangles_and_save_to_file(
    letter_bounding_rectangles, gray, captcha_label
):
    """Saves the individual letters of a CAPTCHA."""
    for letter_bounding_rectangle, current_letter in zip(
        letter_bounding_rectangles, captcha_label
    ):
        letter_image = bounding_rectangle_to_letter_image(
            letter_bounding_rectangle, gray
        )

        save_path = os.path.join(captcha_processing_output_folder, current_letter)
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        character_count = character_counts.get(current_letter, 1)
        np.reshape(letter_image,(-1,1))
        u, count_unique = np.unique(letter_image,return_counts=True)
        if count_unique.size>=10:
            p = os.path.join(save_path, str(character_count) + ".png")
            cv2.imwrite(p, letter_image)
            character_counts[current_letter] = character_count + 1
            
crop_bounding_rectangles_and_save_to_file

<function __main__.crop_bounding_rectangles_and_save_to_file(letter_bounding_rectangles, gray, captcha_label)>

crop_bounding_rectangles_and_save_to_file

In [37]:
import imutils
import numpy as np

for captcha_image_file in captchas:
    captcha_label = get_CAPTCHA_label(captcha_image_file)
    gray, letter_bounding_rectangles = CAPTCHA_to_gray_scale_and_bounding_rectangles(
        captcha_image_file
    )
    
    if len(letter_bounding_rectangles) != 5:
        continue
    
    crop_bounding_rectangles_and_save_to_file(
        letter_bounding_rectangles, gray, captcha_label
    )
    
captchas

['Src_CaptchaIMG\\226md.png',
 'Src_CaptchaIMG\\22d5n.png',
 'Src_CaptchaIMG\\2356g.png',
 'Src_CaptchaIMG\\23mdg.png',
 'Src_CaptchaIMG\\23n88.png',
 'Src_CaptchaIMG\\243mm.png',
 'Src_CaptchaIMG\\244e2.png',
 'Src_CaptchaIMG\\245y5.png',
 'Src_CaptchaIMG\\24f6w.png',
 'Src_CaptchaIMG\\24pew.png',
 'Src_CaptchaIMG\\25257.png',
 'Src_CaptchaIMG\\253dc.png',
 'Src_CaptchaIMG\\25egp.png',
 'Src_CaptchaIMG\\25m6p.png',
 'Src_CaptchaIMG\\25p2m.png',
 'Src_CaptchaIMG\\25w53.png',
 'Src_CaptchaIMG\\264m5.png',
 'Src_CaptchaIMG\\268g2.png',
 'Src_CaptchaIMG\\28348.png',
 'Src_CaptchaIMG\\28x47.png',
 'Src_CaptchaIMG\\2b827.png',
 'Src_CaptchaIMG\\2bg48.png',
 'Src_CaptchaIMG\\2cegf.png',
 'Src_CaptchaIMG\\2cg58.png',
 'Src_CaptchaIMG\\2cgyx.png',
 'Src_CaptchaIMG\\2en7g.png',
 'Src_CaptchaIMG\\2enf4.png',
 'Src_CaptchaIMG\\2fxgd.png',
 'Src_CaptchaIMG\\2g783.png',
 'Src_CaptchaIMG\\2g7nm.png',
 'Src_CaptchaIMG\\2gyb6.png',
 'Src_CaptchaIMG\\2mg87.png',
 'Src_CaptchaIMG\\2mpnn.png',
 'Src_Capt